In [2]:
#=======================================================================================================================================
# Huggingface load_dataset 으로 MLM 훈련 하기
#
# => load_dataset 으로 wiki 말뭉치를 로딩하고, 이를 토크화 시키고, 
# input_ids 에 대해 15% 확률로 [MASK]를 씌워서, 실제 모델을 훈련시키는 예제 
#
# => MLM 훈련 말뭉치는 bongsoo/moco-corpus-kowiki202206 사용, 평가 말뭉치는 bongsoo/bongevalsmall 사용
#
# 출처 : https://wikidocs.net/166817
#=======================================================================================================================================

import torch
import os

from tqdm.notebook import tqdm
from transformers import AutoTokenizer, RobertaTokenizer, RobertaTokenizerFast, RobertaConfig, RobertaModel, RobertaForMaskedLM

import sys
sys.path.append("..")
from myutils import GPU_info, seed_everything, mlogging

# wand 비활성화 
# => trainer 로 훈련시키면 기본이 wandb 활성화이므로, 비활성화 시킴
os.environ["WANDB_DISABLED"] = "true"

In [3]:
# 훈련시킬 말뭉치(사전 만들때 동일한 말뭉치 이용)
#input_corpus = "../../data11/my_corpus/my/pre-kowiki-20220620-1줄.txt"
#input_corpus = "bongsoo/moco-corpus"  # huggingface에 등록된 말뭉치 이용
input_corpus = "../../../korpora/bong_eval.txt"

# eval 말뭉치 
#eval_corpus = "../../data11/my_corpus/bong_small_eval.txt"
eval_corpus = "bongsoo/moco_eval"

# 기존 사전훈련된 모델
model_path = "../../../model/xml-roberta-base"

# 기존 사전 + 추가된 사전 파일
vocab_path = "../../../model/xml-roberta-base/new_bong_token"

# 출력
OUTPATH = '../../../model/xml-roberta-base-bong-1'

############################################################################
# tokenizer 관련 hyper parameter 설정
############################################################################
batch_size = 32       # batch_size
token_max_len = 128   # token_seq_len
epoch = 4             # epoch
lr = 5e-5             # learning rate(기본:5e-5)
############################################################################


device = GPU_info()
print(device)

#seed 설정
seed_everything(333)

#logging 설정
logger =  mlogging(loggername="Roberta-MLM-Trainer", logfilename="../../log/Roberta-MLM-Trainer")

False
device: cpu
cpu
logfilepath:../../log/Roberta-MLM-Trainer_2022-10-04.log


In [4]:
# tokeinzier 생성
# tokenizer 생성
tokenizer = RobertaTokenizerFast.from_pretrained(vocab_path, max_len=token_max_len, do_lower_case=False)
#tokenizer = AutoTokenizer.from_pretrained(vocab_path, max_len=token_max_len, do_lower_case=False)
# fast 토크너나이즈인지 확인
print(f'{vocab_path} is_fast:{tokenizer.is_fast}')

# speical 토큰 계수 + vocab 계수 - 이미 vocab에 포함된 speical 토큰 계수(5)
vocab_size = len(tokenizer.all_special_tokens) + tokenizer.vocab_size - 5 + 1
#vocab_size = len(tokenizer.all_special_tokens) + tokenizer.vocab_size - 5
print('*special_token_size: {}, *tokenizer.vocab_size: {}'.format(len(tokenizer.all_special_tokens), tokenizer.vocab_size))
print('*vocab_size: {}'.format(vocab_size))
print('*tokenizer_len: {}'.format(len(tokenizer)))

# 모델 로딩 further pre-training 
model = RobertaForMaskedLM.from_pretrained(model_path, from_tf=bool(".ckpt" in model_path))   

#################################################################################
# 모델 embedding 사이즈를 tokenizer 크기 만큼 재 설정함.
# 재설정하지 않으면, 다음과 같은 에러 발생함
# CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)` CUDA 에러가 발생함
#  indexSelectLargeIndex: block: [306,0,0], thread: [0,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
#
#     해당 오류는 기존 Embedding(8002, 768, padding_idx=1) 처럼 입력 vocab 사이즈가 8002인데,
#     0~8001 사이를 초과하는 word idx 값이 들어가면 에러 발생함.
#################################################################################
model.resize_token_embeddings(len(tokenizer))

model.to(device)

../../../model/xml-roberta-base/new_bong_token is_fast:True
*special_token_size: 5, *tokenizer.vocab_size: 250002
*vocab_size: 250003
*tokenizer_len: 256542


RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(256542, 768)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=

In [5]:
#==================================================================================================
# load_dataset을 이용하여, 훈련/평가 dataset 로딩.
#
# [로컬 데이터 파일 로딩]
# => dataset = load_dataset("text", data_files='로컬.txt')       # text 로컬 파일 로딩
# => dataset = load_dataset("csv", data_files='로컬.csv')        # csv 로컬 파일 로딩
# => dataset = load_dataset("csv", data_files='로컬.tsv', delimiter="\t")  # tsv 로컬 파일 로딩
# => dataset = load_dataset("json", data_files='로컬.json')      # json 로컬 파일 로딩
# => dataset = load_dataset("pandas", data_files='로컬.pkl')     # pickled dataframe 로컬 파일 로딩
#
# [원격 데이터 파일 로딩]
# url = "https://github.com/crux82/squad-it/raw/master/"
# data_files = {
#    "train": url + "SQuAD_it-train.json.gz",
#    "test": url + "SQuAD_it-test.json.gz",
# }
# squad_it_dataset = load_dataset("json", data_files=data_files, field="data")
#
# 출처 : https://wikidocs.net/166816
#==================================================================================================

from datasets import load_dataset

# 훈련 말뭉치 로딩
#train_dataset = load_dataset(input_corpus)
train_dataset = load_dataset("text", data_files=input_corpus) # text 로컬 파일 로딩

# 평가 말뭉치 로딩
eval_dataset = load_dataset(eval_corpus)

# train_dataset 출력해봄
print(f"train_dataset=======================================")
print(train_dataset)
print(train_dataset['train']['text'][0:3])

print(f'\r\n\r\n')

# eval_dataset 출력해봄
print(f"eval_dataset========================================")
print(eval_dataset)
print(eval_dataset['test']['text'][0:3])

Using custom data configuration default-cb7440ae335106ca


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset text downloaded and prepared to C:/Users/A/.cache/huggingface/datasets/text/default-cb7440ae335106ca/0.0.0/21a506d1b2b34316b1e82d0bd79066905d846e5d7e619823c0dd338d6f1fa6ad. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

...

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset text downloaded and prepared to C:/Users/A/.cache/huggingface/datasets/bongsoo___text/bongsoo--bongevalsmall-5c64781e0e7edefc/0.0.0/21a506d1b2b34316b1e82d0bd79066905d846e5d7e619823c0dd338d6f1fa6ad. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

train_dataset=======================================
DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1500
    })
})
['공공지원 민간임대주택은 주변 시세의 95  이하의 저렴한 임대료 혜택 외에도 품질좋은 주택에서 8년간 안심하고 살 수 있는 장점이 있다 ', '위기아동 발생을 미리 예방할 수 있는 지원활동이나 인프라 구축보다는 피해아동이 발생했을 때 그때그때 대처하는 방식에서 벗어나지 못하고 있다는 것이다 ', '여야가 각종 현안마다 정치적 타협을 모색하는 대신 검찰과 법원을 찾아 상대를 처벌해 달라고 독촉하는  정치의 사법화 가 극심해지고 있다는 지적이다 ']



eval_dataset========================================
DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 200
    })
})
['국토교통부 관계자는  실무기구에서는 각 업계별로 규제혁신형 플랫폼 택시를 하기 위해서는 어떤 규제를 풀어야 한다는 자기 안이 있어야 한다 고 말했다 ', '국책연구기관의 한 관계자는  위원회가 전문성과 대표성을 갖추고 본연의 장점을 최대한 살리기 위해서는 외부 감시와 통제가 보다 활성화돼야 한다 고 지적했다 ', '게임업계 관계자는  현장 수요보다 의료진 등 특정한 누군가의 이익을 위해 게임을 중독물질  질병으로 만들려 한다는 합리적 의심이 든다 고 꼬집었다 ']


In [6]:
# tokenizer 처리
def tokenizer_function(examples):
    result =  tokenizer(examples['text'], truncation=True, max_length=token_max_len, return_overflowing_tokens=True)
    
    # 신규 인덱스와 이전 인덱스와의 매핑 추출
    sample_map = result.pop("overflow_to_sample_mapping")
    for key, values in examples.items():
        result[key] = [values[i] for i in sample_map]
    return result


# batched=True 하면 빠른 tokenizer 이용(Rust)
%time train_dataset_fast = train_dataset.map(tokenizer_function, batched=True)

%time eval_dataset_fast = eval_dataset.map(tokenizer_function, batched=True)

'''
%time tokenized_dataset = text_dataset.map(tokenizer_function, batched=False)
print(tokenized_dataset_fast['train']['text'][0:2])
'''

  0%|          | 0/2 [00:00<?, ?ba/s]

Wall time: 157 ms


  0%|          | 0/1 [00:00<?, ?ba/s]

Wall time: 69.5 ms


"\n%time tokenized_dataset = text_dataset.map(tokenizer_function, batched=False)\nprint(tokenized_dataset_fast['train']['text'][0:2])\n"

In [7]:
print(f"train_dataset_fast=======================================")
print(train_dataset_fast)
print(f'*fast_len:{len(train_dataset_fast["train"])}, len:{len(train_dataset["train"])}')  # fast_dataset과 dataset 길이를 비교함
print(train_dataset_fast['train'][0:2])

print(f'\r\n\r\n')

print(f"eval_dataset_fast=======================================")
print(eval_dataset_fast)
print(f'*fast_len:{len(eval_dataset_fast["test"])}, len:{len(eval_dataset["test"])}')  # fast_dataset과 dataset 길이를 비교함
print(eval_dataset_fast['test'][0:2])

train_dataset_fast=======================================
DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'attention_mask'],
        num_rows: 1500
    })
})
*fast_len:1500, len:1500
{'text': ['공공지원 민간임대주택은 주변 시세의 95  이하의 저렴한 임대료 혜택 외에도 품질좋은 주택에서 8년간 안심하고 살 수 있는 장점이 있다 ', '위기아동 발생을 미리 예방할 수 있는 지원활동이나 인프라 구축보다는 피해아동이 발생했을 때 그때그때 대처하는 방식에서 벗어나지 못하고 있다는 것이다 '], 'input_ids': [[0, 133138, 58646, 209801, 250524, 6, 81956, 697, 61651, 251143, 23406, 9199, 131548, 367, 144975, 252113, 250525, 23855, 20584, 151384, 210859, 250008, 6, 1180, 382, 102030, 250325, 21031, 24777, 1020, 2901, 175554, 469, 3162, 2], [0, 251760, 250326, 39127, 413, 144199, 253020, 8030, 1020, 2901, 20294, 85852, 16632, 250197, 127294, 109355, 253713, 250326, 1504, 39127, 124668, 8978, 252114, 252114, 252115, 10803, 251980, 6, 1180, 223133, 1190, 165061, 53652, 15710, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [8]:
# MLM을 위한 DataCollatorForLangunageModeling 호출
from transformers import DataCollatorForLanguageModeling

# input_ids에 대해 MLM 만들기
data_collator = DataCollatorForLanguageModeling(    # [MASK] 를 씌우는 것은 저희가 구현하지 않아도 됩니다! :-)
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

# input_ids MLM 만들고 출력 해봄
mlm_train_sample = data_collator(train_dataset_fast['train']['input_ids'][0:2])
mlm_eval_sample = data_collator(eval_dataset_fast['test']['input_ids'][0:2])

print(f"train_dataset_fast(MLM)=======================================")
print(mlm_train_sample['input_ids'][0])
print(train_dataset_fast['train'][0])

print(f'\r\n\r\n')

print(f"eval_dataset_fast(MLM)=======================================")
print(mlm_eval_sample['input_ids'][0])
print(eval_dataset_fast['test'][0])

train_dataset_fast(MLM)=======================================
tensor([     0, 133138, 250001, 250001, 250524,      6,  81956,    697,  61651,
        251143,  23406,   9199, 131548,    367, 144975, 252113, 250525,  23855,
         20584, 151384, 250001, 250008,      6,   1180,    382, 250001, 250325,
         21031,  24777,   1020,   2901, 175554,    469, 250001,      2])
{'text': '공공지원 민간임대주택은 주변 시세의 95  이하의 저렴한 임대료 혜택 외에도 품질좋은 주택에서 8년간 안심하고 살 수 있는 장점이 있다 ', 'input_ids': [0, 133138, 58646, 209801, 250524, 6, 81956, 697, 61651, 251143, 23406, 9199, 131548, 367, 144975, 252113, 250525, 23855, 20584, 151384, 210859, 250008, 6, 1180, 382, 102030, 250325, 21031, 24777, 1020, 2901, 175554, 469, 3162, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}



eval_dataset_fast(MLM)=======================================
tensor([     0, 250001,  13916, 250064, 252085,      6,    769, 250137,      6,
        160628,  162

In [9]:
# 훈련 trainer 설정 
# trainer 

from transformers import Trainer, TrainingArguments

#########################################################################################
# hyper parameter 설정
#########################################################################################

epochs = epoch          # epochs

total_optim_steps = len(train_dataset_fast["train"]) * epochs // batch_size   # 총 optimize(역전파) 스탭수 = 훈련dataset 계수 * epochs // 배치 크기
eval_steps=int(total_optim_steps * 0.02)           # 평가 스탭수
logging_steps=eval_steps                           # 로깅 스탭수(*평가스탭수 출력할때는 평가스탭수와 동일하게)
save_steps=int(total_optim_steps * 0.1)            # 저장 스탭수 
save_total_limit=4                                 # 마지막 4개 남기고 삭제 

print(f'*total_optim_steps: {total_optim_steps}, *eval_steps:{eval_steps}, *logging_steps:{logging_steps}, *save_steps:{save_steps}')
#########################################################################################

# cpu 사용이면 'no_cuda = True' 설정함.
no_cuda = False
if device == 'cpu':
    no_cuda = True
print(f'*no_cuda: {no_cuda}')

training_args = TrainingArguments(
    no_cuda = no_cuda,                      # GPU 사용  안함
    output_dir = OUTPATH,                   # 출력 모델 저장 경로 
    overwrite_output_dir=True,         
    num_train_epochs=epochs,                # 에폭
    learning_rate=lr,                      # lr: 기본 5e-5
    per_gpu_train_batch_size=batch_size,    # 배치 사이즈 
    save_steps=save_steps,                  # step 수마다 모델을 저장
    save_total_limit=save_total_limit,      # 마지막 두 모델 빼고 과거 모델은 삭제
    evaluation_strategy="steps",            # 평가 전략 : steps
    eval_steps=eval_steps,                  # 평가할 스텝수
    logging_steps=logging_steps             # 로깅할 스탭수
)

# trainer로 훈련할때는 [mask] 처리된 input_ids 만 dataset으로 넘겨주면 됨.
train_dataset_fast_input_ids = train_dataset_fast['train']['input_ids']
eval_dataset_fast_input_ids = eval_dataset_fast['test']['input_ids']

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,  #MLM(Masked Language Model)
    train_dataset=train_dataset_fast_input_ids,   # 훈련 데이터셋
    eval_dataset=eval_dataset_fast_input_ids      # 평가 데이터셋
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


*total_optim_steps: 187, *eval_steps:3, *logging_steps:3, *save_steps:18
*no_cuda: False


In [10]:
# 훈련 시작
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
F:\AnacondaEnv\daEnv\bong\lib\site-packages\transformers\optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1500
  Num Epochs = 4
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 188


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# 모델 저장
### 전체모델 저장
TMP_OUT_PATH = '../../../model/xml-roberta-base-bong-last'
os.makedirs(TMP_OUT_PATH, exist_ok=True)
#torch.save(model, OUTPATH + 'pytorch_model.bin') 
# save_pretrained 로 저장하면 config.json, pytorch_model.bin 2개의 파일이 생성됨
model.save_pretrained(TMP_OUT_PATH)

# tokeinizer 파일 저장(vocab)
VOCAB_PATH = TMP_OUT_PATH
tokenizer.save_pretrained(VOCAB_PATH)
print(f'==> save_model : {TMP_OUT_PATH}')